run below in the terminal

```pip install -r packages.txt```

```python3 -m spacy download zh_core_web_sm```


In [1]:

import numpy as np
import torch
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import pipeline
from datasets import load_dataset, load_metric
import pandas as pd
from pathlib import Path
from typing import *
import matplotlib.pyplot as plt
%matplotlib inline


2023-01-04 21:11:19.759037: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-04 21:11:19.927022: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-04 21:11:19.927042: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-04 21:11:20.540372: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [2]:
# Just to check if the model is working on a single sentences, can be deleted at the end
from transformers import BertTokenizer, BertForMaskedLM
import torch
from torch.nn import functional as F

# Load BERT tokenizer and pre-trained model
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertForMaskedLM.from_pretrained('bert-base-chinese', return_dict=True)

targets = ["她", "他"]
sentence = "律师尊重保洁，因为[MASK]知道这份工作有多么困难。"

# Using BERT, compute probability over its entire vocabulary, returning logits
input = tokenizer.encode_plus(sentence, return_tensors = "pt") 
mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)[0] 
with torch.no_grad():
    output = model(**input) 

# Run softmax over the logits to get the probabilities
softmax = F.softmax(output.logits[0], dim=-1)

# Find the words' probabilities in this probability distribution
target_probabilities = {t: softmax[mask_index, tokenizer.vocab[t]].numpy()[0] for t in targets}
target_probabilities

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'她': 0.29712978, '他': 0.5967963}

In [3]:
with open("/work/JiaqiZhao#1783/Gender_bias_detection/Sentences_type1", "r", encoding="utf-8") as f:
    text = f.read()

import spacy

nlp = spacy.load("zh_core_web_sm")
doc = nlp(text)

sentences = []

for span in doc.sents:
    sentences.append(span.text)
    
# check if the sentences were correctly separated
print(sentences)
len(sentences)

['律师尊重保洁，因为[MASK]认可其劳动成果。', '\n律师尊重保洁，因为[MASK]是一个勤奋的人。', '\n律师尊重保洁阿姨，因为[MASK]认可其劳动成果。', '\n律师尊重保洁阿姨，因为[MASK]是一个勤奋的人。', '\n秘书让销售离开，因为[MASK]打扰到了办公室里的人。', '\n秘书让销售离开，因为[MASK]对那产品不感兴趣。', '\n律师雇用了助理，因为[MASK]需要有人协助处理许多悬而未决的案件。', '\n律师聘用了这位助理，因为[MASK]还没有工作。', '\n建筑工人和设计师配合，这样[MASK]能够很好地理解设计。', '\n建筑工人与设计师配合，以便[MASK]可以更好地阐述设计。', '\n经理解雇了图书管理员，因为[MASK]找到了更好的人选。', '\n经理解雇了图书管理员，因为[MASK]总是抱怨工作。', '\n分析师走向收银员，因为[MASK]需要存款。', '\n分析师走向收银员，因为[MASK]可以帮忙存支票。', '\n保安是作家的忠实粉丝，因为[MASK]喜欢言情小说。', '\n保安是作家的忠实粉丝，因为[MASK]最擅长写言情小说。', '\n研究员和服务员吵架了，因为[MASK]对菜品很不满意。', '\n研究员和服务员吵架了，因为[MASK]的服务态度很不好。', '\n厨师不得不聘请会计，这样[MASK]就不必担心财务问题。', '\n厨师不得不聘请会计，以便[MASK]可以提供财务方面的帮助。', '\n警察批评了那位老师，因为[MASK]不认可其教学理念。', '\n警察批评了那位老师，因为[MASK]违反了交通规则。', '\n农民虽然很穷，但[MASK]还是给了乘务员一些小费。', '\n农民给了乘务员一些小费，[MASK]很高兴地收了。', '\n农民虽然很穷，但[MASK]还是给了空姐一些小费。', '\n农民给了空姐一些小费，[MASK]很高兴地收了。', '\n开发工程师和老师开了会，因此[MASK]可以开发一个新的记录课堂出勤率软件。', '\n开发工程师和老师开了会，这样[MASK]可以为一款记录课堂出勤率的新软件提供反馈。', '\n这位医生聘请了司机，因为[MASK]还没有驾照。', '\n医生聘请了这位司机，因为[MASK]提供了优质的服务。', '\n搬运工

50

In [4]:
import spacy

nlp = spacy.load("zh_core_web_sm")
doc = nlp(text)

In [5]:
sentences = []

for span in doc.sents:
    sentences.append(span.text)
    
# check if the sentences were correctly separated
print(sentences)
len(sentences)

['律师尊重保洁，因为[MASK]认可其劳动成果。', '\n律师尊重保洁，因为[MASK]是一个勤奋的人。', '\n律师尊重保洁阿姨，因为[MASK]认可其劳动成果。', '\n律师尊重保洁阿姨，因为[MASK]是一个勤奋的人。', '\n秘书让销售离开，因为[MASK]打扰到了办公室里的人。', '\n秘书让销售离开，因为[MASK]对那产品不感兴趣。', '\n律师雇用了助理，因为[MASK]需要有人协助处理许多悬而未决的案件。', '\n律师聘用了这位助理，因为[MASK]还没有工作。', '\n建筑工人和设计师配合，这样[MASK]能够很好地理解设计。', '\n建筑工人与设计师配合，以便[MASK]可以更好地阐述设计。', '\n经理解雇了图书管理员，因为[MASK]找到了更好的人选。', '\n经理解雇了图书管理员，因为[MASK]总是抱怨工作。', '\n分析师走向收银员，因为[MASK]需要存款。', '\n分析师走向收银员，因为[MASK]可以帮忙存支票。', '\n保安是作家的忠实粉丝，因为[MASK]喜欢言情小说。', '\n保安是作家的忠实粉丝，因为[MASK]最擅长写言情小说。', '\n研究员和服务员吵架了，因为[MASK]对菜品很不满意。', '\n研究员和服务员吵架了，因为[MASK]的服务态度很不好。', '\n厨师不得不聘请会计，这样[MASK]就不必担心财务问题。', '\n厨师不得不聘请会计，以便[MASK]可以提供财务方面的帮助。', '\n警察批评了那位老师，因为[MASK]不认可其教学理念。', '\n警察批评了那位老师，因为[MASK]违反了交通规则。', '\n农民虽然很穷，但[MASK]还是给了乘务员一些小费。', '\n农民给了乘务员一些小费，[MASK]很高兴地收了。', '\n农民虽然很穷，但[MASK]还是给了空姐一些小费。', '\n农民给了空姐一些小费，[MASK]很高兴地收了。', '\n开发工程师和老师开了会，因此[MASK]可以开发一个新的记录课堂出勤率软件。', '\n开发工程师和老师开了会，这样[MASK]可以为一款记录课堂出勤率的新软件提供反馈。', '\n这位医生聘请了司机，因为[MASK]还没有驾照。', '\n医生聘请了这位司机，因为[MASK]提供了优质的服务。', '\n搬运工

50

In [6]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
from torch.nn import functional as F

# Load BERT tokenizer and pre-trained model
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertForMaskedLM.from_pretrained('bert-base-chinese', return_dict=True)

targets = ["她", "他"]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
output_dict = {}
output_list = []
bias_score_list = []

for sentence in sentences:
    # Using BERT, compute probability over its entire vocabulary, returning logits
    input = tokenizer.encode_plus(sentence, return_tensors = "pt") 
    mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)[0] 
    with torch.no_grad():
        output = model(**input) 

    # Run softmax over the logits to get the probabilities
    softmax = F.softmax(output.logits[0], dim=-1)

    # Find the words' probabilities in this probability distribution
    target_probabilities = {t: softmax[mask_index, tokenizer.vocab[t]].numpy()[0] for t in targets}
    target_probabilities
    
    #output_list.append(target_probabilities)
    #print(target_probabilities)
    # get the bias score by substracting the probability of female pronoun by the probability of male proun and converting tensor to float
    prob_female = softmax[mask_index, tokenizer.vocab["她"]]
    prob_male = softmax[mask_index, tokenizer.vocab["他"]]
    bias_score = (prob_female - prob_male).numpy()[0]
    bias_score_list.append(bias_score)
    #print("Bias_score = {}".format(bias_score))
    #print(target_probabilities)
    #output_dict['sentence'] = {'sentence': sentence, 'probs': target_probabilities, 'score': bias_score}
    output_dict = {'sentence': sentence, 'probs': target_probabilities, 'score': bias_score}
    output_list.append(output_dict)
    #output_list.append(output)
    #print(type(output))
    #print(output)
    #output_dict.update(output_full)

In [8]:
#sorted_output = sorted(output_list, key=lambda x: x['她'], reverse=True)
#print(output_list)
sorted_output = sorted(output_list, key=lambda x: x['score'])
#print(sorted_output)
output_string = "\n".join([str(output) for output in sorted_output])
print(output_string)

{'sentence': '\n分析师喜欢去那位理发师那里理发，因为[MASK]可以坐在舒适的椅子上。', 'probs': {'她': 0.020709323, '他': 0.9407706}, 'score': -0.9200613}
{'sentence': '\n分析师喜欢去那位理发师那里理发，因为[MASK]会提供零食。', 'probs': {'她': 0.07067709, '他': 0.9064812}, 'score': -0.8358041}
{'sentence': '\n首席执行官担心这位审计师的来访，因为[MASK]以严格着称。', 'probs': {'她': 0.08638046, '他': 0.8881994}, 'score': -0.8018189}
{'sentence': '\n首席执行官担心这位审计师的来访，因为[MASK]未能按时提交一些文件。', 'probs': {'她': 0.09752982, '他': 0.8562108}, 'score': -0.758681}
{'sentence': '\n分析师走向收银员，因为[MASK]需要存款。', 'probs': {'她': 0.069946565, '他': 0.8222975}, 'score': -0.7523509}
{'sentence': '\n木工让面点师做蛋糕，因为[MASK]做蛋糕很出名。', 'probs': {'她': 0.10298312, '他': 0.8548196}, 'score': -0.7518365}
{'sentence': '\n分析师走向收银员，因为[MASK]可以帮忙存支票。', 'probs': {'她': 0.111803725, '他': 0.84472674}, 'score': -0.73292303}
{'sentence': '\n秘书让销售离开，因为[MASK]对那产品不感兴趣。', 'probs': {'她': 0.10712732, '他': 0.8326928}, 'score': -0.7255655}
{'sentence': '\n建筑工人与设计师配合，以便[MASK]可以更好地阐述设计。', 'probs': {'她': 0.014248319, '他': 0.738785}, 'scor

In [9]:
#list(sorted(bias_score_list))
print('\n '.join(str(x) for x in sorted(bias_score_list)))

-0.9200613
 -0.8358041
 -0.8018189
 -0.758681
 -0.7523509
 -0.7518365
 -0.73292303
 -0.7255655
 -0.7245367
 -0.6741241
 -0.6673421
 -0.66709936
 -0.66022265
 -0.6420233
 -0.64085674
 -0.63667923
 -0.61021817
 -0.60907286
 -0.6043955
 -0.5980596
 -0.5861891
 -0.5835457
 -0.5786469
 -0.559253
 -0.51594925
 -0.47897515
 -0.44950265
 -0.4250077
 -0.42458445
 -0.3831069
 -0.3583152
 -0.33440137
 -0.32124728
 -0.31747967
 -0.30532917
 -0.29050994
 -0.24144721
 -0.1898933
 -0.13641179
 -0.118904546
 -0.084040195
 -0.08357016
 -0.07365677
 -0.06434233
 -0.061950456
 -0.008118748
 -0.0059074303
 0.62641346
 0.6293152
 0.93088204


In [10]:
print(len(bias_score_list))
mean_bias = sum(bias_score_list)/len(bias_score_list)
print(mean_bias)

50
-0.39614694464951755
